# 一等函数
## 5.1 把函数视为对象

In [1]:
def fac(n):
    '''return n!'''
    return 1 if n < 2 else n * fac(n-1)
fac(42)

1405006117752879898543142606244511569936384000000000

In [2]:
fac.__doc__

'return n!'

In [3]:
type(fac)

function

In [4]:
f = fac
f

<function __main__.fac(n)>

In [5]:
f(5)

120

In [6]:
list(map(f, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

## 5.2高阶函数

In [7]:
alpha = ['abcd', 'efg', 'hijkl', 'mnopqr', 'stuvwxyz']
sorted(alpha, key = len)

['efg', 'abcd', 'hijkl', 'mnopqr', 'stuvwxyz']

In [8]:
def reverse(word):
    return word[::-1]
reverse('apple')

'elppa'

In [9]:
sorted(alpha, key = reverse)

['abcd', 'efg', 'hijkl', 'mnopqr', 'stuvwxyz']

#### map,filter,reduce的现代替代品

In [10]:
list(map(fac, range(6)))

[1, 1, 2, 6, 24, 120]

In [11]:
[fac(n) for n in range(6)]

[1, 1, 2, 6, 24, 120]

In [12]:
list(map(fac, filter(lambda n: n%2, range(6))))

[1, 6, 120]

In [13]:
[fac(n) for n in range(6) if n%2]

[1, 6, 120]

In [15]:
from functools import reduce
from operator import add
reduce(add, range(120))

7140

In [16]:
sum(range(120))

7140

## 5.3 匿名函数

In [17]:
sorted(alpha, key = lambda w: w[::-1])

['abcd', 'efg', 'hijkl', 'mnopqr', 'stuvwxyz']

## 5.4可调用函数

In [18]:
callable(abs)

True

In [19]:
callable(reverse)

True

## 5.5 用户定义可调用对象

In [22]:
import random

class BingoCage:
    
    def __init__(self, items):
        self._items = items
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        return self.pick()

In [25]:
bingo = BingoCage(list(range(10)))
bingo.pick()

0

In [26]:
bingo()

2

## 5.6函数内省

In [27]:
dir(fac)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [28]:
fac.__dict__

{}

In [33]:
class C: 
    pass

obj = C()
def f(): pass
set(dir(f)) - set(dir(obj))

{'__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__'}

## 5.7从定位参数到仅限关键字参数

In [45]:
def tag(name, *content, cls = None, **attrs):
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join('%s = "%s"'%(attr, value) for attr, value in sorted(attrs.items()))
    
    else:
        attr_str = ''
        
    if content:
        return '\n'.join("<%s%s>%s</%s>"%(name, attr_str, c, name) for c in content)
    else:
        return'<%s%s/>'%(name, attr_str)

In [38]:
tag('br')

'<br/>'

In [39]:
tag('p', 'hello')

'<p>hello</p>'

In [46]:
print(tag('p', 'hello', 'world', cls = 's'))

<pclass = "s">hello</p>
<pclass = "s">world</p>


In [48]:
print(tag('p', 'hello', 'world', 's'))

<p>hello</p>
<p>world</p>
<p>s</p>


## 5.8获取关于参数的信息

In [50]:
!pip install bobo

  Created wheel for bobo: filename=bobo-2.4.0-cp37-none-any.whl size=17333 sha256=4d320d9c54b9db2b8cba46eeebc309035d6986884764849a7aeccffd23ff6fe5
  Stored in directory: C:\Users\renzi\AppData\Local\pip\Cache\wheels\a1\31\84\614c9a3cd82ec8300be51f46a21a1985a785da734d5138a5a2
Successfully built bobo


In [51]:
import bobo
@bobo.query('/')
def hello(person):
    return 'Hello %s'%person


In [60]:
def clip(text, max_len = 80):
    end = None
    if len(text)> max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
        if space_after >= 0:
            end = space_after
        if end is None:
            end = len(text)
        return text[:end].rstrip()

In [61]:
clip.__defaults__

(80,)

In [57]:
from clip import clip

In [59]:
clip.__defaults__

(80,)

In [62]:
clip.__code__.co_varnames

('text', 'max_len', 'end', 'space_before', 'space_after')

In [63]:
clip.__code__.co_argcount

2

In [64]:
from inspect import signature
sig = signature(clip)

In [65]:
str(sig)

'(text, max_len=80)'

In [67]:
for name, para in sig.parameters.items():
    print(para.kind, ':', name ,'=', para.default)

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


In [68]:
import inspect
sig = inspect.signature(tag)
my_tag = {
    'name':'img',
    'title':'Sunset',
    'src':'sunset.jpg',
    'cls':'framed'
}

bound_arg = sig.bind(**my_tag)
bound_arg

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset', 'src': 'sunset.jpg'})>

In [69]:
for name, value in bound_arg.arguments.items():
    print(name, '=', value)

name = img
cls = framed
attrs = {'title': 'Sunset', 'src': 'sunset.jpg'}


## 5.9 函数注解

In [ ]:
def clip(text:str, max_len:'int > 0' = 80)->str:
    end = None
    if len(text)> max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
        if space_after >= 0:
            end = space_after
        if end is None:
            end = len(text)
        return text[:end].rstrip()

## 5,10支持函数式编程的包
### 5.10.1operator模块

In [72]:
from functools import reduce
def fac(n):
    return reduce(lambda a, b: a*b, range(1, n+1))
fac(11)

39916800

In [73]:
from functools import reduce
from operator import mul
def fac2(n):
    return reduce(mul, range(1, n+1))
fac2(11)

39916800

In [74]:
from operator import methodcaller
s = 'The time has come'
upcase = methodcaller('upper')
upcase(s)

'THE TIME HAS COME'

### 5.10.2 使用functools.partial冻结参数

In [75]:
from operator import mul
from functools import partial
triple = partial(mul, 3)
triple(7)

21

In [76]:
list(map(triple, range(10)))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

In [78]:
import unicodedata
nfc = partial(unicodedata.normalize, 'NFC')
nfc('cafe\u0301')

'café'